### Extract from GPT

In [94]:
import pandas as pd
from icecream import ic
import re, os
from tqdm import tqdm

def extract_text_generated_by_gpt(project, original_csv, to_save_csv, run, gpt_variant, flag_file):
    original_df = pd.read_csv(original_csv)
    bug_ids = []
    summaries, descriptions = [], []
    all_bugs = set()
    found_bugs = set()
    modified_bug_count = 0
    mixed_keywords = 0
    flag_df = pd.read_csv(flag_file)
    skipped_bug_ids = flag_df[flag_df['run_flag'] == 0]['bug_id'].tolist()
    for i, bug_id in enumerate(skipped_bug_ids):
        skipped_bug_ids[i] = str(bug_id)
    skipped_bug_ids = set(skipped_bug_ids)
    
    for i, row in original_df.iterrows():
        bug_id = int(row['bug_id'])        
        
        if not os.path.exists(f'../data/keywords/{project}/{gpt_variant}/run_{run}/{bug_id}.txt'):
            summaries.append(original_df.loc[i, 'short_desc'])
            descriptions.append(original_df.loc[i, 'description'])
            bug_ids.append(bug_id)
            continue
        
        if str(bug_id) in skipped_bug_ids:
            summaries.append(original_df.loc[i, 'short_desc'])
            descriptions.append(original_df.loc[i, 'description'])
            bug_ids.append(bug_id)
            continue
        
        with open(f'../data/keywords/{project}/{gpt_variant}/run_{run}/{bug_id}.txt', 'r') as f:
            lines = f.readlines()
            
        all_bugs.add(bug_id)            
        rephrased_summary, rephrased_description = '', ''
        found_response = False
        found_summary_description = False
        response_begin = -1
        
        for line_index, line in zip(range(len(lines)), lines):
            if '>>>>>> Response:' in line:    
                found_response = True
                response_begin = line_index + 1
                continue
            
            if found_response:
                summary_match = re.match(r'^Summary:\s*(?:\[)?(.*?)(?:\])?$', line.strip())
                
                if summary_match:
                    rephrased_summary = summary_match.group(1)
                
                if len(rephrased_summary) > 0:
                    description_match = re.match(r'^Description:\s*(?:\[)?(.*?)(?:\])?$', line.strip())
                    if description_match:
                        rephrased_description = description_match.group(1)
                        summaries.append(rephrased_summary)
                        descriptions.append(rephrased_description)
                        bug_ids.append(bug_id)
                        found_bugs.add(bug_id)
                        rephrased_summary = ''
                        rephrased_description = ''
                        found_response = False
                        modified_bug_count += 1
                        found_summary_description = True
                        break
        
        if not found_summary_description and found_response:
            # os.remove(f'../data/keywords/{project}/{gpt_variant}/run_{run}/{bug_id}.txt') 
            # ic(response_begin)        
            # print(f'No summary and description found for bug {bug_id}')
            # Regular expression to match each line starting with a hyphen
            regex = r"-\s*(.+)"
            mixed_keywords += 1
            for line_idx in range(response_begin, len(lines)):
                line = lines[line_idx]
                matches = re.findall(regex, line)
                if len(matches) > 0:                    
                    rephrased_summary += matches[0] + ' '                
                    rephrased_description += matches[0] + ' '
                                    
            # ic(bug_id)
            # ic(rephrased_summary, rephrased_description)
            bug_ids.append(bug_id)
            summaries.append(rephrased_summary)
            descriptions.append(rephrased_description)
            rephrased_summary = ''
            rephrased_description = ''
            found_response = False
            modified_bug_count += 1
            found_bugs.add(bug_id)
        
        found_response = False
        found_summary_description = False
        
        bug_dict = dict()
        for bug in bug_ids:
            bug_dict[bug] = bug_dict.get(bug, 0) + 1
        for bug in bug_dict:
            if bug_dict[bug] == 2:
                print(bug)
            if bug_dict[bug] == 0:
                print(bug)
    ic(run)
    ic(all_bugs - found_bugs)
    ic(len(bug_ids), len(summaries), len(descriptions))
    ic(modified_bug_count)
    ic(mixed_keywords)
    # pd.DataFrame({
    #     'bug_id': bug_ids, 
    #     'short_desc': summaries, 
    #     'description': descriptions
    # }).to_csv(to_save_csv)
    
project = 'spark'
gpt_variant = 'gpt_verbose'

for run in range(1, 6):
    extract_text_generated_by_gpt(project, \
        f'../data/raw/test_{project}.csv', \
        f'../data/keywords/{project}/{gpt_variant}_test_{run}.csv', \
        run, \
        gpt_variant, \
        f'../data/ablation/test_{project}_flag_content.csv', \
    )

ic| run: 1
ic| all_bugs - found_bugs: set()
ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841
ic| modified_bug_count: 1466
ic| mixed_keywords: 248
ic| run: 2
ic| all_bugs - found_bugs: set()
ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841
ic| modified_bug_count: 1466
ic| mixed_keywords: 247
ic| run: 3
ic| all_bugs - found_bugs: set()
ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841
ic| modified_bug_count: 1466
ic| mixed_keywords: 249
ic| run: 4
ic| all_bugs - found_bugs: set()
ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841
ic| modified_bug_count: 1466
ic| mixed_keywords: 254
ic| run: 5
ic| all_bugs - found_bugs: set()
ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841
ic| modified_bug_count: 1466
ic| mixed_keywords: 241


### Extract from LLMs

In [47]:
import pandas as pd
from icecream import ic
import re, os
from tqdm import tqdm

def extract_text_generated_by_llm(project, original_csv, to_save_csv, run, llm):
    original_df = pd.read_csv(original_csv)
    bug_ids = []
    summaries, descriptions = [], []
    all_bugs = set()
    found_bugs = set()
    modified_bug_count = 0
        
    for i, row in original_df.iterrows():
        bug_id = int(row['bug_id'])
        # if i == 5:
        #     break
        
        if not os.path.exists(f'../data/keywords/{project}/{llm}/run_{run}/{bug_id}.txt'):
            summaries.append(original_df.loc[i, 'short_desc'])
            descriptions.append(original_df.loc[i, 'description'])
            bug_ids.append(bug_id)
            continue
        
        with open(f'../data/keywords/{project}/{llm}/run_{run}/{bug_id}.txt', 'r') as f:
            lines = f.readlines()
            
        all_bugs.add(bug_id)            
        rephrased_summary, rephrased_description = '', ''
        found_response = False
        found_summary_description = False
        response_begin = -1
        
        for line_index, line in zip(range(len(lines)), lines):
            if '>>>>>> Response:' in line:    
                found_response = True
                response_begin = line_index + 1
                continue
            
            if found_response:
                summary_match = re.match(r'^Summary:\s*(?:\[)?(.*?)(?:\])?$', line.strip())
                summary_star_match = re.match(r'^\*\*Summary:\*\*\s*(?:\[)?(.*?)(?:\])?$', line.strip())
                

                if summary_match:
                    rephrased_summary = summary_match.group(1)
                    # ic(rephrased_summary)
                if summary_star_match:
                    rephrased_summary = summary_star_match.group(1)
                    # ic(rephrased_summary)
                    
                if len(rephrased_summary) > 0:
                    description_match = re.match(r'^Description:\s*(?:\[)?(.*?)(?:\])?$', line.strip())
                    description_star_match = re.match(r'^\*\*Description:\*\*\s*(?:\[)?(.*?)(?:\])?$', line.strip())
                    if description_match or description_star_match:                                               
                        if description_match:
                            rephrased_description = description_match.group(1)
                        elif description_star_match:
                            rephrased_description = description_star_match.group(1)
                        summaries.append(rephrased_summary)
                        descriptions.append(rephrased_description)
                        # ic(rephrased_summary, rephrased_description)
                        bug_ids.append(bug_id)
                        found_bugs.add(bug_id)
                        rephrased_summary = ''
                        rephrased_description = ''
                        found_response = False                        
                        modified_bug_count += 1
                        found_summary_description = True
                        break
                    
        if not found_summary_description and found_response:
            # os.remove(f'../data/keywords/{project}/{llm}/run_{run}/{bug_id}.txt')         
            # print(f'No summary and description found for bug {bug_id}')
            regex = r"-\s*(.+)"
            found_summary_again = False
            found_description_again = False
            for line_idx in range(response_begin, len(lines)):
                line = lines[line_idx]
                if 'Summary:' in line:
                    found_summary_again = True
                    continue
                if 'Description:' in line:
                    found_description_again = True
                    break
                    
            if found_summary_again and found_description_again:            
                description_begin = response_begin + 1
                for line_idx in range(response_begin, len(lines)):
                    line = lines[line_idx]
                    if 'Description:' in line:
                        description_begin = line_idx + 1
                        break
                    matches = re.findall(regex, line)
                    if len(matches) > 0:
                        rephrased_summary += matches[0] + ' '
                
                for line_idx in range(description_begin, len(lines)):
                    line = lines[line_idx]
                    matches = re.findall(regex, line)
                    if len(matches) > 0:
                        rephrased_description += matches[0] + ' '
            else:
                for line_idx in range(response_begin, len(lines)):
                    line = lines[line_idx]
                    matches = re.findall(regex, line)
                    if len(matches) > 0:
                        rephrased_summary += matches[0] + ' '
                        rephrased_description += matches[0] + ' '
                                    
            # ic(bug_id)
            # ic(rephrased_summary, rephrased_description)
            bug_ids.append(bug_id)
            summaries.append(rephrased_summary)
            descriptions.append(rephrased_description)
            rephrased_summary = ''
            rephrased_description = ''
            found_response = False
            modified_bug_count += 1
            found_bugs.add(bug_id)
        
        found_response = False
        found_summary_description = False
                                                               
        bug_dict = dict()
        for bug in bug_ids:
            bug_dict[bug] = bug_dict.get(bug, 0) + 1
        for bug in bug_dict:
            if bug_dict[bug] == 2:
                print(bug)
            if bug_dict[bug] == 0:
                print(bug)
    ic(run)
    ic(all_bugs - found_bugs)
    ic(len(bug_ids), len(summaries), len(descriptions))
    ic(modified_bug_count)
    
    pd.DataFrame({
        'bug_id': bug_ids, 
        'short_desc': summaries, 
        'description': descriptions
    }).to_csv(to_save_csv)

project = 'spark'
llm = 'llama3'

for run in range(1, 2):
    extract_text_generated_by_llm(project, \
        f'../data/raw/test_{project}.csv', \
        f'../data/keywords/{project}/{llm}_test_{run}.csv', \
        run, \
        llm
    )

ic| run: 1
ic| all_bugs - found_bugs: set()
ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841
ic| modified_bug_count: 1466


### Filtering

In [3]:
from tqdm import tqdm
import json
import numpy as np

def investigate_data(project):
    desc_lens = []
    
    with open('../data/raw/test_{}.txt'.format(project), 'r') as f:
        test_data = f.readlines()
    ic(len(test_data[1].split()))
    test_bug_ids = set()
    
    for bug_id in test_data[1].split():
        test_bug_ids.add(bug_id)
    print(len(test_bug_ids))
    
    original_content_json = '../data/raw/{}.json'.format(project)
    with open(original_content_json, 'r') as f:
        original_content_lines = f.readlines()
    descriptions = []
    
    for line in tqdm(original_content_lines):
        bug = json.loads(line)        
        if bug['bug_id'] in test_bug_ids:
            continue
        
        descriptions.append(bug['description'])
        # ic(bug['description'])
        # ic(bug['bug_id'], len(bug['description'].split()))
        desc_lens.append(len(bug['description'].split()))
    
    p25 = np.percentile(desc_lens, 25)
    p50 = np.percentile(desc_lens, 50)
    p75 = np.percentile(desc_lens, 75)
    ic(project)
    ic(p25, p50, p75)
    # ic(long_count)
    # Spark: ic| p25: 29.0, p50: 61.0, p75: 125.0
    # Hadoop: ic| p25: 24.0, p50: 48.0, p75: 94.0
    # Kibana: ic| p25: 41.0, p50: 74.0, p75: 132.0

for project in ['spark', 'hadoop', 'kibana']:
    investigate_data(project)


ic| len(test_data[1].split()): 2841


2841


100%|██████████| 9579/9579 [00:00<00:00, 98161.33it/s]
ic| project: 'spark'
ic| p25: 29.0, p50: 61.0, p75: 125.0
ic| len(test_data[1].split()): 3740


3740


100%|██████████| 14016/14016 [00:00<00:00, 66900.98it/s]
ic| project: 'hadoop'
ic| p25: 24.0, p50: 48.0, p75: 94.0
ic| len(test_data[1].split()): 7167


7167


100%|██████████| 17016/17016 [00:00<00:00, 87954.98it/s]
ic| project: 'kibana'
ic| p25: 41.0, p50: 74.0, p75: 132.0


### Filter Length

In [4]:
def filter_length(project, length_threshold):
    """
    Ablation study 1: filter out the bug reports whose description 
    length is less than the threshold
    """
    
    df = pd.read_csv('../data/raw/test_{}.csv'.format(project), index_col=None)
    bug_ids = []
    run_flag = []
    for i, row in df.iterrows():
        bug_id = row['bug_id']
        bug_ids.append(bug_id)
        if len(str(row['description']).split()) < length_threshold:
            run_flag.append(0)
            continue
        run_flag.append(1)
    df = pd.DataFrame({'bug_id': bug_ids, 'run_flag': run_flag})
    print(project)
    print(df['run_flag'].value_counts())
    df.to_csv('../data/ablation/test_{}_flag_length.csv'.format(project))

filter_length('spark', 125)
filter_length('hadoop', 94)
filter_length('kibana', 132)

spark
run_flag
0    2255
1     586
Name: count, dtype: int64
hadoop
run_flag
0    2837
1     903
Name: count, dtype: int64
kibana
run_flag
0    5284
1    1883
Name: count, dtype: int64


### Filter Content

In [80]:
import re

def contains_code_url(s):
    patterns = {
        "code_items": r"\w+\.\w+\.\w{1,}",
        "urls": r"https?://\S+"
    }
    
    # Correctly join the values from the patterns dictionary
    pattern = re.compile('|'.join(patterns.values()))

    # Search for any matches in the string
    if pattern.search(s):
        return True
    else:
        return False

    
def filter_content(project):
    """
    Ablaion study 2: filter out the bug reports whose description does not contain
    URLs or code
    """
    df = pd.read_csv('../data/raw/test_{}.csv'.format(project), index_col=None)
    bug_ids = []
    run_flag = []
    
    for i, row in df.iterrows():
        bug_id = row['bug_id']
        bug_ids.append(bug_id)
        if contains_code_url(str(row['description'])):
            run_flag.append(1)
            continue       
        run_flag.append(0)
    
    df = pd.DataFrame({'bug_id': bug_ids, 'run_flag': run_flag})
    print(project)
    print(df['run_flag'].value_counts())
    df.to_csv('../data/ablation/test_{}_flag_content.csv'.format(project))

# for project in ['spark', 'hadoop', 'kibana']:
#     filter_content(project)
contains_code_url('com.example.MyClass.myMethod(MyClass.java:42)')

True

In [23]:
def check_llm_generated_data(project, llm, flag_file):
    bug_ids = set()
    flag_df = pd.read_csv(flag_file)
    for i, row in flag_df.iterrows():
        if row['run_flag'] == 1:
            bug_ids.add(row['bug_id'])
    for bug_id in bug_ids:
        if not os.path.exists(f'../data/keywords/{project}/{llm}/run_1/{bug_id}.txt'):
            print(bug_id)
            continue
check_llm_generated_data('spark', 'openchat', '../data/ablation/test_spark_flag_content.csv')

13320672
13313818
13347610
13315043
13316584
13316594
13284984
13285010
13285017
13294066
13294068
13278803
13328229


### Filter Length Content

In [51]:
def filter_length_content(project, length_threshold):
    """
    Ablation study 3: filter the bug reports whose description length is greater than or equal to the threshold
    or contains code
    """
    df = pd.read_csv('../data/raw/test_{}.csv'.format(project), index_col=None)
    ic(df.shape)
    bug_ids = []
    run_flag = []
    
    for i, row in df.iterrows():
        bug_id = row['bug_id']
        bug_ids.append(bug_id)
        if len(str(row['description']).split()) >= length_threshold:
            run_flag.append(1)
            continue
        if contains_code_url(str(row['description'])):
            run_flag.append(1)
            continue
        run_flag.append(0)
    
    df = pd.DataFrame({'bug_id': bug_ids, 'run_flag': run_flag})
    print(project)
    print(df['run_flag'].value_counts())
    df.to_csv(f'../data/ablation/test_{project}_flag_length_content.csv')

# filter_length('spark', 125)
# filter_length('hadoop', 94)
# filter_length('kibana', 132)

filter_length_content('spark', 125)
filter_length_content('hadoop', 93)
filter_length_content('kibana', 132)

ic| df.shape: (2841, 3)
ic| df.shape: (3740, 3)


spark
run_flag
1    1583
0    1258
Name: count, dtype: int64


ic| df.shape: (7167, 3)


hadoop
run_flag
0    1895
1    1845
Name: count, dtype: int64
kibana
run_flag
1    3791
0    3376
Name: count, dtype: int64


### Extract from Keywords

In [89]:
import pandas as pd
from icecream import ic
import re, os

def extract_text_generated_by_keyword_extractor(project, original_csv, to_save_csv, run, keyword_extractor):
    
    original_df = pd.read_csv(original_csv)
    bug_ids = []
    summaries, descriptions = [], []
    
    for i, row in original_df.iterrows():
        
        bug_id = row['bug_id']
        
        if not os.path.exists('../data/keywords/{}/{}/run_{}/{}.txt'.format(project, keyword_extractor, run, bug_id)):
            summaries.append(original_df.loc[i, 'short_desc'])
            descriptions.append(original_df.loc[i, 'description'])
            bug_ids.append(bug_id)
            continue
            
        with open('../data/keywords/{}/{}/run_{}/{}.txt'.format(project, keyword_extractor, run, bug_id), 'r') as f:
            lines = f.readlines()
        
        rephrased_summary, rephrased_description = '', ''
        
        for line in lines:
            if "Summary:" in line:
                rephrased_summary = line.split('Summary:')[1].strip()
                continue
                
            if "Description:" in line:
                rephrased_description = line.split('Description:')[1].strip()
                                 
                bug_ids.append(bug_id)
                summaries.append(rephrased_summary)
                descriptions.append(rephrased_description)
                # ic(bug_id, rephrased_summary, rephrased_description)
                                                             
        bug_dict = dict()
        for bug in bug_ids:
            bug_dict[bug] = bug_dict.get(bug, 0) + 1
        for bug in bug_dict:
            if bug_dict[bug] == 2:
                print(bug)
    
    ic(len(bug_ids), len(summaries), len(descriptions))
    
    pd.DataFrame({
        'bug_id': bug_ids, 
        'short_desc': summaries, 
        'description': descriptions
    }).to_csv(to_save_csv)
    
keyword_extractor = 'kpminer-idf'
run = 1

project = 'spark'
extract_text_generated_by_keyword_extractor(
    project, \
    '../data/raw/test_{}.csv'.format(project), \
    '../data/keywords/{}/{}_test_{}.csv'.format(project, keyword_extractor, run), \
    run, keyword_extractor)

ic| len(bug_ids): 2841, len(summaries): 2841, len(descriptions): 2841


### Write Back filtering

In [90]:
# !pip install ujson
import ujson

from tqdm import tqdm

def write_back_json_filtering(run, flag_file, to_save_json, project, keyword_extractor):
    with open(f'../data/raw/test_{project}.txt', 'r') as f:
        test_data = f.readlines()
    summary_keywords_count = []
    run_ids = []
    cannot_produce = 0
    description_keywords_count = []
    test_bug_ids = set()
    for bug_id in test_data[1].split():
        test_bug_ids.add(bug_id)        
    ic(len(test_bug_ids))
    
    original_content_json = '../data/raw/{}.json'.format(project)
    df = pd.read_csv(f'../data/keywords/{project}/{keyword_extractor}_test_{run}.csv')
    
    flag_df = pd.read_csv(flag_file)
    skipped_bug_ids = flag_df[flag_df['run_flag'] == 0]['bug_id'].tolist()
    for i, bug_id in enumerate(skipped_bug_ids):
        skipped_bug_ids[i] = str(bug_id)
    skipped_bug_ids = set(skipped_bug_ids)
    
    with open(original_content_json, 'r') as f:
        original_content_lines = f.readlines()
    
    output_lines = []
    modified_bug_id_count = 0
    for line in tqdm(original_content_lines):
        bug = ujson.loads(line)
        
        if bug['bug_id'] in skipped_bug_ids:
            output_lines.append(ujson.dumps(bug))
            continue
        
        if bug['bug_id'] in test_bug_ids:
            if not os.path.exists(f'../data/keywords/{project}/{keyword_extractor}/run_{run}/{bug["bug_id"]}.txt'):
                print(bug['bug_id'])
                cannot_produce += 1
                continue
            try:
                row = df[df['bug_id'] == int(bug['bug_id'])].iloc[0]
            except IndexError:
                print(type(bug['bug_id']))
                print(bug['bug_id'])
                
            bug['short_desc'] = row['short_desc']
            summary_keywords_count.append(len(str(row['short_desc']).split()))
            bug['description'] = row['description']
            description_keywords_count.append(len(str(row['description']).split()))
            modified_bug_id_count += 1
            run_ids.append(bug['bug_id'])
        
        output_lines.append(ujson.dumps(bug))
    
    with open(to_save_json, 'w') as f:
        for bug in output_lines:
            f.write(bug)
            f.write('\n')
            
    ic(modified_bug_id_count)
    ic(cannot_produce)
    print('Average summary keywords count:', round(sum(summary_keywords_count) / len(summary_keywords_count), 1))
    print('Average description keywords count:', round(sum(description_keywords_count) / len(description_keywords_count), 1))
    for i, desc_len in zip(range(len(description_keywords_count)), description_keywords_count):
        if desc_len > 1000:
            print(run_ids[i], i, desc_len)
            


keyword_extractor = 'kpminer-idf'
project = 'spark'

for run in range(1, 2):
    # write_back_json_filtering(run, \
    # f'../data/ablation/test_{project}_flag_length.csv', \
    # f'../data/ablation/{project}_gpt_length_r{run}.json', \
    #     project)
    
    write_back_json_filtering(run, \
    f'../data/ablation/test_{project}_flag_content.csv', \
    f'../data/ablation/{project}_{keyword_extractor}_content_r{run}.json', \
    project, \
    keyword_extractor)
    
    # write_back_json_filtering(run, \
    # f'../data/ablation/test_{project}_flag_length_content.csv', \
    # f'../data/ablation/{project}_gpt_length_content_r{run}.json', \
    # project,
    # keyword_extractor)

ic| len(test_bug_ids): 2841
100%|██████████| 9579/9579 [00:00<00:00, 18695.55it/s]
ic| modified_bug_id_count: 1466
ic| cannot_produce: 0


Average summary keywords count: 11.0
Average description keywords count: 13.9


In [ ]:
from tqdm import tqdm
import ujson

def write_back_json(project, flag_file, run, to_save_json):
    with open('../data/raw/test_{}.txt'.format(project), 'r') as f:
        test_data = f.readlines()        
    test_bug_ids = set()
    for bug_id in test_data[1].split():
        test_bug_ids.add(bug_id)
    ic(len(test_bug_ids))
    
    original_content_json = '../data/raw/{}.json'.format(project)
    df = pd.read_csv('../data/keywords/{}/{}_test_{}.csv'.format(project, keyword_extractor, run))
    
    with open(original_content_json, 'r') as f:
        original_content_lines = f.readlines()
    
    flag_df = pd.read_csv(flag_file)
    skipped_bug_ids = flag_df[flag_df['run_flag'] == 0]['bug_id'].tolist()
    for i, bug_id in enumerate(skipped_bug_ids):
        skipped_bug_ids[i] = str(bug_id)
    skipped_bug_ids = set(skipped_bug_ids)
    
    output_lines = []
    for line in tqdm(original_content_lines):
        bug = ujson.loads(line)
            
        if bug['bug_id'] in test_bug_ids:
            try:
                row = df[df['bug_id'] == int(bug['bug_id'])].iloc[0]
            except IndexError:
                print(type(bug['bug_id']))
                print(bug['bug_id'])
            bug['short_desc'] = row['short_desc']
            bug['description'] = row['description']
        
        output_lines.append(ujson.dumps(bug))
    
    with open(to_save_json, 'w') as f:
        for bug in output_lines:
            f.write(bug)
            f.write('\n')
      
keyword_extractor = 'tfidf'
project = 'spark'
run = 1
write_back_json(project, run, '../data/keywords/{}/{}_{}_p1_r{}.json'.format(project, project, keyword_extractor, run))
# for run in range(1, 6):
    # write_back_json(project, run, '../data/keywords/{}/{}_{}_p1_r{}.json'.format(project, project, keyword_extractor, run))

### Comparing Two Runs

In [86]:
import filecmp
import os

def compare_files(folder1, folder2, filename):
    """
    Check whether two files are the same.
    """
    
    file1 = folder1 + '/' + filename
    file2 = folder2 + '/' + filename
    
    if not filecmp.cmp(file1, file2):        
    #     print(f"The files {file1} and {file2} are different.")
        return 1
    return 0
        
# folder1 = '../data/keywords/spark/llama3/run_3'

folder1 = '../data/keywords/spark/llama3/run_1'
folder2 = '../data/keywords/spark/llama3/run_2'
# folder2 = '../data/keywords/spark/llama3-new/run_1'

total_diff = 0
for file in os.listdir(folder2):
    total_diff += compare_files(folder1, folder2, file)
ic(total_diff)


ic| total_diff: 0


0

In [45]:
hadoop_length_flag = pd.read_csv('../data/ablation/test_hadoop_flag_length.csv')
hadoop_content_flag = pd.read_csv('../data/ablation/test_hadoop_flag_content.csv')
hadoop_length_content_flag = pd.read_csv('../data/ablation/test_hadoop_flag_length_content.csv')
length_flag_1 = set(hadoop_length_flag[hadoop_length_flag['run_flag'] == 1]['bug_id'].tolist())
content_flag_1 = set(hadoop_content_flag[hadoop_content_flag['run_flag'] == 1]['bug_id'].tolist())
length_content_flag_1 = set(hadoop_length_content_flag[hadoop_length_content_flag['run_flag'] == 1]['bug_id'].tolist())
ic(len(length_flag_1), len(content_flag_1), len(length_content_flag_1))
ic(length_content_flag_1 - content_flag_1)

ic| len(length_flag_1): 31
    len(content_flag_1): 62
    len(length_content_flag_1): 63
ic|

 length_content_flag_1 - content_flag_1: {13307183}


{13307183}